In [ ]:
import json 
import pickle 
import nltk
import numpy as np

from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from nltk.stem import SnowballStemmer
import random


CONSTRUCCIÓN DE LA BIBLIOTECA DE INTENCIONES 

In [ ]:
def guardar_json(datos,filename):
    archivo=open(filename,'w')
    json.dump(datos,archivo,indent=4)

In [ ]:
biblioteca = {"intents": 
              
              [{"tag":"saludos",
                 "patterns":['hola','holi', 'como vas?',
                            'buenos dias','que pasa?',
                            'buenas tardes','como andas?','alo','tengo una duda','una duda','duda','dudita',
                            'como estas?','buena','buen dia','consulta','consultita',
                            'tengo una consulta','buenas noches','tengo una dudita',
                            'necesito ayuda', 'hello','que tal?','necesito que me ayudes','una consulta',
                            'que me puedes ofrecer?','buenas',
                            'que haces?','holis','holu','que hay?'],

                 "responses":['''Hola, soy Rockie!! un asistente virtual, en qué puedo ayudarte? 
                                 Si desea conocer lo que puedo ofrecer envíe *1* '''],

                 "context":[""]
                 },    
                 
                 {"tag":"despedidas",
                 "patterns":['chao','adios','hasta luego','nos vemos','bye','chaito','hasta la proxima','hasta nunca','hasta la vista'],
                "responses":['Hasta luego que tenga un buen dia',' Gracias por preferirnos!','Hasta la proxima, tenga buen dia'],
                "context":[],
                 },
                 
                
                 {"tag":"agradecimientos",
                 "patterns":["gracias","muchisimas gracias","me ayudaste mucho","fuiste de ayuda",
                             "muchas gracias","demasiado util","dale, gracias",
                             "mil gracias",
                             "muy amable",
                             "se lo agradezco",
                             "fue de ayuda",
                             "gracias por la ayuda",
                             "muy agradecido",
                             "gracias por su tiempo"                             
                            ],

                 "responses":["De nada",
                              "Feliz por ayudarlo",
                              "Gracias a usted",
                              "Estamos para servirle",
                              "Fue un placer", "A la orden","Con gusto",
                              "Un gusto comunicarme contigo"
                             ],
                 "context":[""]
                },

                {
                    "tag": "Afirmación",
                    "patterns": [
                        "Si",'', 'Sí', 'Asi mismito','asi mismito','ecolecua', 'Exacto','Es correcto', 'Listo', 'Perfecto', 'Maravilloso', 'genial', 'excelente', 'correcto', 'Ok', 'Está bien', 'esta bien' ],
                    "responses": [""],
                    "context": [""]
                },

                 {
                    "tag": "Negación",
                    "patterns": [
                    "No","Es incorrecto","No, gracias","No gracias","No me gusta","Es erroneo","Tiene un error","Falta","incorrecto","Asi no","Esta mal","Falso","No es cierto"
                    ],
                    "responses": ["" ],
                    "context": [""]
                },
                 
                 
                 
                 {"tag":"Agendar cita",
                 "patterns":["Quisiera agendar una cita",
                             "Puedo agendar una cita?","Agendar cita"
                             'Quiero agendar cita',
                             "Quiero agendar una cita"                          
                            ],
                 "responses":["""'Para agendar su cita coloque el número de unas de las modalidades disponible: 1. Virtual 2. Presencial'
                                 """
                             ],
                 "context":[""]
                },
                 
               
                {"tag":"Modificar Cita",
                 "patterns":["Quiero modificar mi cita",
                             "Puedo modificar mi cita?","Modificame la cita","Modifica","Puedes modificarla?","quiero que la cambies",
                             "Modificar cita","Modifica la cita","Modificar","Cambia la cita","Cambiame la cita",
                             "Me puedes cambiar la cita?","Cambiar fecha de la cita","Cambiar hora de la cita",
                             "Modificar hora de la cita","Modificar fecha de la cita"                       
                            ],
                 "responses":["""Para modificar su cita ingrese el id dado al momento de generar su cita.
                                  """
                             ],
                 "context":[""]
                },
               
               
               
                {"tag":"Cancelar Cita",
                 "patterns":["Quiero cancelar mi cita",
                             "Puedo cancelar mi cita?","Me cancelas la cita","Cancelar","Cancela la cita",
                             "Cancelar cita","Mejor cancelala","Cancela mejor","cancela","Cancelala","Cancelalo"                         
                            ],
                 "responses":[""" Para modificar su cita ingrese el id de cita dado al momento de generar su cita
                                  """
                             ],
                 "context":[""]
                },

                {"tag":"Garantia",
                 "patterns":["Tienen garantia?","Garantia del producto?","Que periodo de garantia tiene?","Cuanto de garantia tiene?",
                             "Cuanto dura su garantia?","Cuanta garantia tiene?","Esto que garantia tiene?","Cuanto dura la garantia?",
                             "Garantia","Mi pedido vino defectuoso","El producto vino estropeado","Quiero reclamar","Reclamar","Reclamo","Devolucion","Devolver","Quiero devolver","Voy a devolverlo"                         
                            ],
                 "responses":[""" Todo nuestros productos tienen una garantia de un año si quiere hacer un reclamo puede llamar a nuestro soporte
                                  """
                             ],
                 "context":[""]
                },

                {"tag":"Pagos",
                 "patterns":["Como puedo pagar?","Como pago?","Donde pago?","Donde le pago?","Como le pago?","A donde paso a pagarle?",
                             "Puedo pagar con paypal?","El pago?","A donde voy a pagar?","A donde me dirijo a pagar?",
                             "Formas de pago","Como puedo hacer el pago","Pagar"                          
                            ],
                 "responses":[""" En nuestra empresa contamos con diferentes formas de pago puede utilizar la que más comoda le parezca:
                 -PayPal
                 -Tarjetas visa ó mastercard
                 -Tarjetas de regalo
                                  """
                             ],
                 "context":[""]
                },
                {"tag":"Ubicación",
                 "patterns":["Ubicación", "Donde los encuentro?","Donde los puedo encontrar","Donde esta el local?","Direccion del local?",
                             "Dónde se ubican?","Direccion","Indicaciones de como llegar","Indicaciones","Como llego alla?","Como llego?",
                             "Tienen un local físico?","Mandeme la ubicacion","A que ubicacion voy?","A que lugar exacto tengo que ir?",
                             "A donde tengo que ir?","Me pasas la ubicacion","Me pasas la direccion","Me manda la ubicacion "
                            ],
                 "responses":[""" Estamos ubicado en Costa del Este, Ciudad de Panamá, Panamá
                                   Aquí el link de nuestra ubicación exacta: 
                                  """
                             ],
                 "context":[""]
                },
                 
                 
                {"tag":"norespuesta",
                 "patterns":[""],
                 "responses":["No puedo reponder a tu consulta, podrías ser más claro?",
                              "Disculpa no entiendo tu solicitud, podrías ser más claro?"
                             ],
                 "context":[""]                    
                },

                {
                    "tag": "Servicio",
                    "patterns": [
                        "Que tipo de servicio ofrecen?","Que hace su local?","Que tienen en ese local?",
                        "Servicios","Que ofrecen?","Que servicios tienen?","Digame los servicios","Que hay en la tienda?",
                        "Que hacen?",
                        'A qué se dedican'
                    ],
                    "responses": [
                        """Nuestra empresa SIC-Panamá se encarga de transportar cualquier producto que necesites a nivel mundial, 
                            para bridarle a nuestros clientes contamos con un conjunto de herramientas que le permita escoger la 
                            opción que más se adaptan a ellos, como asesorías directa o online,cotización de precios y mucho más"""
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "Confrontación con el chatbot",
                    "patterns": [
                        "Eres un chatbot",
                        "Estoy escribiendole a un agente virtual?",
                        "Eres una persona real?"
                    ],
                    "responses": [
                    "Soy un agente virtual construido con tecnología"
                    ],
                    "context": [
                        ""
                    ]
                },

                {
                    "tag": "Cotización de precios",
                    "patterns": [
                        "Quiero cotizar precios","Puedes hacer la cotizacion?","Haz la cotizacion",
                        "Deseo cotizar precios","Cotizame los precios","Cotiza","Cotizame","Cotizar"
                        "Cotizar precios","Cotizacion","Puedes cotizarme los precios?"
                    ],
                    "responses": [
                        "Para cotizar precios, indiquenos el paquete que desea. Si desea conocer cuáles osn envíe *1*"
                    ],
                    "context": [
                        ""
                    ]
                },

                { "tag": "Menú de opciones",
                    "patterns": [
                        ""
                    ],
                    "responses": [
                        """1. Cita de asesoría
                           2. Cotizar precios
                           3. Envío de paquetes
                           4. Preguntas frecuentes"""
                    ],
                    "context": [""]
                }
                ]            
                }
                

# Guardado de diccionario de conocimiento en formato json.
guardar_json(biblioteca, 'intents.json')

In [ ]:
bolsaPalabras=[]
clases=[]
documents=[]

for intent in biblioteca['intents']:

    clases.append(intent['tag'])


    for pattern in intent['patterns']:
        result=nltk.word_tokenize(pattern)
        bolsaPalabras.extend(result)
        documents.append((result,intent['tag']))




USO DE LA LEMITIZACIÓN Y LIMPIEZA DE LA BOLSA DE PALABRA

In [ ]:
stemmer=SnowballStemmer('spanish')

ignore_words= ["?","¿","!","¡","."] 

bolsadepalabras = [stemmer.stem(w.lower()) for w in bolsaPalabras if w not in ignore_words]

 OBTENER DATOS DE ENTRENAMIENTO 

In [ ]:
def cleanString(words,ignore_words):
    words=[stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    return words

bolsaPalabras=cleanString(bolsaPalabras,ignore_words)
pickle.dump(bolsaPalabras, open('bolsaPalabras.pkl','wb'))
pickle.dump(clases, open('clases.pkl','wb'))
training= []

for doc in documents:

    interaccion=doc[0]
    interaccion=cleanString(interaccion,ignore_words)

    entradacodificada=[]

    for palabra in bolsaPalabras:
        if palabra in interaccion:
            entradacodificada.append(1)
        else:
            entradacodificada.append(0)

    salidacodificada=[0]*len(clases)
    indice=clases.index(doc[1])
    salidacodificada[indice]=1

    training.append([entradacodificada,salidacodificada])

training=np.array(training,dtype=list)

x_train=list(training[:,0])

y_train=list(training[:,1])





CREAR REDES NEURONALES

In [ ]:
model=Sequential()

#model.add(Dense(256,input_shape=(len(x_train[0]),),activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

ENTRENAR MODELO

In [ ]:
sgd = SGD(learning_rate=0.01,momentum=0.9,nesterov=True) # ,decay=1e-6
model.compile(loss="categorical_crossentropy", optimizer=sgd,metrics=["accuracy"])
hist = model.fit(np.array(x_train),np.array(y_train),epochs=300,batch_size=5,verbose=True)
model.save('bot_model.h5',hist)



###CORRER DESPUÉS DE COLOCAR TODAS LAS INTENCIONES 

PREPARAR FUNCIONES DE RESPUESTAS DEL CHATBOT

In [ ]:
ignore_words= ["?","¿","!","¡","."]
model=load_model('bot_model.h5')
biblioteca=json.loads(open('intents.json').read())
bolsaPalabras=pickle.load(open('bolsaPalabras.pkl','rb'))
clases=pickle.load(open('clases.pkl','rb'))

In [ ]:
def limpiarentrada(entradaUsuario):
    entradaUsuario = nltk.word_tokenize(entradaUsuario)
    entradaUsuario = [stemmer.stem(w.lower()) for w in entradaUsuario if w not in ignore_words]
    return entradaUsuario

def convVector(entradaUsuario, bolsaPalabras):
    
    entradaUsuario = limpiarentrada(entradaUsuario)
    
    vectorentrada = [0]*len(bolsaPalabras)    
    for palabra in entradaUsuario:              
        
        if palabra in bolsaPalabras:         
            
            indice = bolsaPalabras.index(palabra)   
            vectorentrada[indice] = 1                  
            
    vectorentrada = np.array(vectorentrada)            
    return vectorentrada


In [ ]:
def obt_tag(vectorentrada,LIMITE=0):
    vector_salida=model.predict(np.array([vectorentrada]))[0]

    vector_salida=[[i,r] for i,r in enumerate(vector_salida) if r > LIMITE]

    vector_salida.sort(key=lambda x: x[1],reverse=True)
    print(vector_salida)

    list_etiquetas=[]

    for r in vector_salida:
        list_etiquetas.append({'intent':clases[r[0]],'probability':str(r[1])})
   
    return list_etiquetas



In [ ]:

def obt_respuesta(list_etiquetas,biblioteca):
    etiqueta=list_etiquetas[0]['intent']

    lista_diccionarios=biblioteca['intents']

    for diccionario in lista_diccionarios:

        if etiqueta==diccionario['tag']:
            list_respuestas=diccionario['responses']
            respuesta=random.choice(list_respuestas)

            break
    
    return respuesta


In [ ]:
def chatbot_respuesta(entradaUsuario):
    vector_entrada=convVector(entradaUsuario,bolsaPalabras)
    list_etiquetas=obt_tag(vector_entrada,LIMITE=0)
    respuesta=obt_respuesta(list_etiquetas,biblioteca)
    intencion=list_etiquetas[0]['intent']
    return respuesta,intencion

In [ ]:
entradaUsuario = ''
intencion_previa = ''
msgconf = 'Para terminar el proceso de agendar su cita, confirme que los datos ingresados son correctos.'

while entradaUsuario != 'exit':
    entradaUsuario = input()
    respuesta, intencion = chatbot_respuesta(entradaUsuario)
    intencion_previa=intencion
    print(respuesta)